In [1]:
import pymongo
import pprint

In [2]:
M = pymongo.MongoClient()

In [3]:
distortions = M.ferroelectric_dataset.distortions
workflow_data = M.ferroelectric_dataset.workflow_data

In [4]:
smooth = workflow_data.find({'polarization_len' : 10,
                             'polarization_max_spline_jumps': {"$exists": True, "$not" : { "$gt" : 1 }},
                             'energies_per_atom_max_spline_jumps': {'$lte': 1e-2}})
not_smooth = workflow_data.find({'polarization_len' : 10,
                                 'polarization_change_norm': {'$exists': True},
                                 '$or': [{'polarization_max_spline_jumps': { "$gt" : 1 }},
                                         {'energies_per_atom_max_spline_jumps': {'$gt': 1e-2}}]})
static = workflow_data.find({'$or': [{'polarization_len' : {'$lt': 10}},
                                     {'polarization_change_norm': {'$exists': False}}],
                             'static_len': 10,
                             'workflow_status': 'COMPLETED'})
defused_or_fizzled_or_running = workflow_data.find(
    {'$and': [{'$or': [{'polarization_len' : {'$lt': 10}},
                       {'polarization_change_norm': {'$exists': False}}]},
              {'$or': [{'workflow_status': 'DEFUSED'}, 
                       {'workflow_status': 'FIZZLED'}, 
                       {'workflow_status': 'RUNNING'}]}]})
print("smooth: ", smooth.count())
print("not_smooth: ", not_smooth.count())
print("static_only: ", static.count())
print("not_complete: ", defused_or_fizzled_or_running.count())

smooth:  184
not_smooth:  55
static_only:  42
not_complete:  133


/home/tess/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
/home/tess/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
/home/tess/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
/home/tess/.local/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


In [5]:
def get_alphabetical_formula(pretty_formula):
    from pymatgen.core.composition import Composition
    alphabetical_formula = Composition(pretty_formula).alphabetical_formula.split()
    # Remove 1s
    alphabetical_formula = [string[:-1] if (string[-1] == "1" and string[-2] not in "0123456789") else string for string in alphabetical_formula]
    # Stringify
    alphabetical_formula = "".join(alphabetical_formula)
    return alphabetical_formula

In [6]:
def make_csv_table(pymongo_result, filename):
    array = ['Workflow id', 'Formula', 'Polar id', 'Nonpolar id', 
             'Polar Spacegroup', 'Nonpolar Spacegroup', 'Polarization', 
             'Energy Difference', 'Max Distortion', 'Band Gap']
    with open(filename, 'w') as f:
        f.write(",".join(array) + "\n")
        for entry in pymongo_result:
            array = ['<a href="candidate.html?wfid=' + entry['wfid'] + '">'  + entry['wfid'] + '</a>',
#                      entry['pretty_formula'],
                     get_alphabetical_formula(entry['pretty_formula']),
                     '<a href="https://materialsproject.org/materials/' + entry['polar_id'] +\
                     '/">' + entry['polar_id'] + '</a>',
                     '<a href="https://materialsproject.org/materials/' + entry['nonpolar_id'] +\
                     '/">' + entry['nonpolar_id'] + '</a>',
                     int(entry['polar_spacegroup']),
                     int(entry['nonpolar_spacegroup']),
                     "--" if 'polarization_change_norm' not in entry else "%.1f" % float(entry['polarization_change_norm']),
                     "--" if 'energies_per_atom' not in entry or len(entry['energies_per_atom']) < 2 else "%.3f" % (float(entry['energies_per_atom'][0]) - float(entry['energies_per_atom'][-1])),
                     "--" if 'calculated_max_distance' not in entry else "%.3f" % float(entry['calculated_max_distance']),
                     "--" if 'bandgaps' not in entry else "%.3f" % entry['bandgaps'][-1]]
            f.write(",".join(map(str,array)) + "\n")

In [7]:
filenames = ["../csv/smooth_table_data.csv", "../csv/not_smooth_table_data.csv", 
             "../csv/static_only_table_data.csv", "../csv/not_complete_table_data.csv"]
pymongo_results = [smooth, not_smooth, static, defused_or_fizzled_or_running]

In [8]:
for filename, pymongo_result in zip(filenames, pymongo_results):
    make_csv_table(pymongo_result, filename)